In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define CNN model class
def create_model(num_conv_layers, filter_size):
    layers = []
    in_channels = 1  # Fashion-MNIST has 1 channel

    for i in range(num_conv_layers):
        out_channels = filter_size * (2 ** i)
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        in_channels = out_channels

    layers.append(nn.Flatten())

    # Dynamically compute the input size for the first Linear layer
    dummy_input = torch.zeros(1, 1, 28, 28)  # Assuming input size 28x28
    with torch.no_grad():
        for layer in layers:
            dummy_input = layer(dummy_input) if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)) else dummy_input
    flatten_size = dummy_input.numel()

    layers.append(nn.Linear(flatten_size, 128))
    layers.append(nn.ReLU())
    layers.append(nn.Linear(128, 10))

    return nn.Sequential(*layers)

# Train function
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Test function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root="./data", train=False, transform=transform, download=True)

# Hyperparameter combinations
configs = [
    {"num_conv_layers": 2, "filter_size": 16, "learning_rate": 0.001, "batch_size": 32},
    {"num_conv_layers": 2, "filter_size": 32, "learning_rate": 0.001, "batch_size": 64},
    {"num_conv_layers": 3, "filter_size": 16, "learning_rate": 0.001, "batch_size": 32},
    {"num_conv_layers": 3, "filter_size": 32, "learning_rate": 0.001, "batch_size": 64},
    {"num_conv_layers": 2, "filter_size": 16, "learning_rate": 0.0005, "batch_size": 32},
    {"num_conv_layers": 2, "filter_size": 32, "learning_rate": 0.0005, "batch_size": 64},
    {"num_conv_layers": 3, "filter_size": 16, "learning_rate": 0.0005, "batch_size": 32},
    {"num_conv_layers": 3, "filter_size": 32, "learning_rate": 0.0005, "batch_size": 64},
    {"num_conv_layers": 4, "filter_size": 16, "learning_rate": 0.001, "batch_size": 32},
    {"num_conv_layers": 4, "filter_size": 32, "learning_rate": 0.001, "batch_size": 64},
]

# Results storage
results = []

# Experiment with configurations
for idx, config in enumerate(configs):
    print(f"Running configuration {idx + 1}/{len(configs)}: {config}")

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    # Create model
    model = create_model(config["num_conv_layers"], config["filter_size"]).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

    # Train the model
    for epoch in range(5):  # Train for 5 epochs per configuration
        train_loss = train_model(model, train_loader, criterion, optimizer)
        print(f"Epoch [{epoch + 1}/5], Loss: {train_loss:.4f}")

    # Test the model
    accuracy = test_model(model, test_loader)
    print(f"Accuracy: {accuracy:.2f}%\n")

    # Store results
    results.append({"config": config, "accuracy": accuracy})

# Print summary of results
print("Experiment Results:")
for result in results:
    print(f"Config: {result['config']}, Accuracy: {result['accuracy']:.2f}%")


# Config: {'num_conv_layers': 4, 'filter_size': 32, 'learning_rate': 0.001, 'batch_size': 64}
# best accuracy: Accuracy: 91.67%